In [1]:
DATASET_PATH="/home/ll16598/Documents/INNOVATE"
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
#import ruptures as rpt
import warnings
#import ruptures
#import requests
import time
import pickle
import networkx as nx
from tqdm import tqdm
warnings.filterwarnings('ignore')


In [2]:
df=pd.read_csv('./Data/Derivative_Datasets/jump_data.csv')
paper_titles = list(set(df['paper_title']))

In [3]:
with open('./Data/papers-with-abstracts.json') as f:
    pwc=json.load(f)
title_to_abstract = {item['title']: item['abstract'] for item in pwc}
len(title_to_abstract.values())
pwc=None

In [4]:
with open('./Data/paper-code-links.json') as f:
    paper_code=json.load(f)    



In [5]:
len(paper_code)

256224

In [6]:
import os
import re
import requests
from github import Github
import pprint
import gzip

# Optionally, set your GitHub access token for higher rate limits:
access_token = ""  # Replace with your token or None
g = Github(access_token) if access_token else Github()

# Sample paper_code list for demonstration (your actual list may be larger)

def get_repo_from_url(repo_url):
    """
    Extracts the owner and repository name from a GitHub repo URL.
    Example: 'https://github.com/user/repo' -> ('user', 'repo')
    """
    pattern = r"github\.com/([^/]+)/([^/]+)"
    match = re.search(pattern, repo_url)
    if match:
        owner, repo_name = match.groups()
        # Remove any trailing characters such as '.git'
        repo_name = repo_name.replace('.git', '')
        return owner, repo_name
    return None, None

def get_all_python_files(repo, extensions=(".py", ".ipynb")):
    """
    Recursively retrieves all files with the specified extensions in the given repository.
    Returns a list of dictionaries with file information.

    Parameters:
        repo: The PyGithub Repository object.
        extensions (tuple): A tuple of file extensions to include.
                            Defaults to (".py", ".ipynb").

    Returns:
        List[dict]: A list of dictionaries containing file information.
                    Each dictionary has keys: "path", "download_url", and "name".
    """
    files_found = []
    contents = repo.get_contents("")
    
    while contents:
        file_content = contents.pop(0)
        if file_content.type == "dir":
            contents.extend(repo.get_contents(file_content.path))
        else:
            if file_content.name.endswith(extensions):
                files_found.append({
                    "path": file_content.path,
                    "download_url": file_content.download_url,
                    "name": file_content.name
                })
    return files_found


In [7]:
def sanitize_filename(filename):
    # Replace any character that is not alphanumeric, underscore, dash, or space with an underscore.
    return re.sub(r'[^\w\-_ ]', '_', filename)


In [8]:
completed_titles=os.listdir('/media/ll16598/One Touch/INNOVATE/pwc_python_files_from_git/')

In [9]:
len(completed_titles)

22925

In [ ]:
import os
import re
import requests
import pickle
import gc  # Import garbage collector

def build_folder_tree(python_files):
    tree = {}
    for file_info in python_files:
        rel_path = file_info.get("path", file_info["name"])
        parts = rel_path.split('/')
        current_level = tree
        for part in parts[:-1]:
            current_level = current_level.setdefault(part, {})
        current_level.setdefault("files", []).append(parts[-1])
    return tree

base_download_dir = "/media/ll16598/One Touch/INNOVATE/pwc_python_files_from_git/"

paper_python_files = {}
paper_folder_structures = {}

for e, entry in enumerate(paper_code):
    repo_url = entry.get('repo_url')
    paper_title = entry.get('paper_title', 'unknown_paper')
    if sanitize_filename(paper_title).replace(" ", "_") in completed_titles:
        continue
    completed_titles.append(paper_title)
    if not repo_url:
        print(f"No repo_url found for paper: {paper_title}")
        continue
    
    owner, repo_name = get_repo_from_url(repo_url)
    if not owner or not repo_name:
        print(f"Could not parse repo_url: {repo_url}")
        continue

    try:
        repo = g.get_repo(f"{owner}/{repo_name}")
        python_files = get_all_python_files(repo)
        paper_python_files[paper_title] = python_files
        print(f"Found {len(python_files)} Python files in repository '{repo_name}' for paper '{paper_title}'.")

        folder_structure = build_folder_tree(python_files)
        paper_folder_structures[paper_title] = folder_structure

        safe_title = sanitize_filename(paper_title).replace(" ", "_")
        paper_dir = os.path.join(base_download_dir, safe_title)
        os.makedirs(paper_dir, exist_ok=True)
    
        for file_info in python_files:
            file_url = file_info["download_url"]
            rel_path = file_info.get("path", file_info["name"])
            file_path = os.path.join(paper_dir, rel_path)
            os.makedirs(os.path.dirname(file_path), exist_ok=True)
            
            try:
                response = requests.get(file_url)
                response.raise_for_status()
                with open(file_path, "w", encoding="utf-8") as f:
                    f.write(response.text)
            except Exception as e:
                pass
            finally:
                # Important: close the response to release its connection/ buffers
                response.close()
                # Delete the response object so it can be garbage-collected
                del response

        # Pickle updates (still optional, but presumably you want to do it here)
        with open(os.path.join(base_download_dir, f'paper_python_links_{e}.pickle'), 'wb') as f:
            pickle.dump(paper_python_files, f)
        with open(os.path.join(base_download_dir, f'paper_folder_structures_{e}.pickle'), 'wb') as f:
            pickle.dump(paper_folder_structures, f)

    except Exception as e:
        print(f"Error processing repo {repo_url}: {e}")
    
    # Force a collection at the end of each main loop iteration
    gc.collect()


Error processing repo https://github.com/h3nok/OCM: 404 {"message": "Not Found", "documentation_url": "https://docs.github.com/rest/repos/repos#get-a-repository", "status": "404"}
Could not parse repo_url: https://worksheets.codalab.org/worksheets/0xe3eb416773ed4883bb737662b31b4948
Could not parse repo_url: https://worksheets.codalab.org/worksheets/0xecc9a01cfcbc4cd6b0444a92d259a87c
Error processing repo https://github.com/KishiKyle/KishiKyle.github.io: 404 {"message": "Not Found", "documentation_url": "https://docs.github.com/rest/repos/repos#get-a-repository", "status": "404"}
Could not parse repo_url: https://gitlab.physik.uni-muenchen.de/tk-svm/tksvm-op
Could not parse repo_url: https://gitlab.physik.uni-muenchen.de/tk-svm/tksvm-op
Could not parse repo_url: https://gitlab.physik.uni-muenchen.de/tk-svm/tksvm-op
Could not parse repo_url: https://gitlab.physik.uni-muenchen.de/tk-svm/tksvm-op
Could not parse repo_url: https://gitlab.com/maximelenormand/mismatch-connectivity
Could not p

Following Github server redirection from /repos/gillesvntnu/gcn_multistructure to /repositories/696275958


Found 64 Python files in repository 'gcn_multistructure' for paper 'Towards Robust Cardiac Segmentation using Graph Convolutional Networks'.
Error processing repo https://github.com/gillesvntnu/gcn_multistructure: 'list' object has no attribute 'setdefault'
Error processing repo https://github.com/large-ocr-model/large-ocr-model.github.io: 404 {"message": "Not Found", "documentation_url": "https://docs.github.com/rest/repos/repos#get-a-repository", "status": "404"}
Error processing repo https://github.com/mzakariaCERN/mzakaria.github.io: 404 {"message": "Not Found", "documentation_url": "https://docs.github.com/rest/repos/repos#get-a-repository", "status": "404"}
Error processing repo https://github.com/iceplussss/QSAR-Complete: 404 {"message": "Not Found", "documentation_url": "https://docs.github.com/rest/repos/repos#get-a-repository", "status": "404"}
Could not parse repo_url: https://plmlab.math.cnrs.fr/stoch-algo-phys/generative-models/fixed-kinetic-NHF/
Could not parse repo_url: 

Request GET /repos/vivek3141/ghostbuster-data/contents/reuter/gpt_semantic/EricAuchard failed with 403: Forbidden
Setting next backoff to 255.19024s


Found 0 Python files in repository 'ghostbuster-data' for paper 'Ghostbuster: Detecting Text Ghostwritten by Large Language Models'.
Found 4 Python files in repository 'react-emus' for paper 'Stage-IV Cosmic Shear with Modified Gravity and Model-independent Screening'.
Found 23 Python files in repository 'vfl-admm' for paper 'Improving Privacy-Preserving Vertical Federated Learning by Efficient Communication with ADMM'.
Found 125 Python files in repository 'OSFNTC' for paper 'An Open-Source Framework for Efficient Numerically-Tailored Computations'.
Found 0 Python files in repository 'DiffSCI' for paper 'DiffSCI: Zero-Shot Snapshot Compressive Imaging via Iterative Spectral Diffusion Model'.
Found 4 Python files in repository 'traffic_dynamics' for paper 'Spatiotemporal Implicit Neural Representation as a Generalized Traffic Data Learner'.
Found 6 Python files in repository 'llm-ell' for paper 'A Law of Next-Token Prediction in Large Language Models'.
Found 0 Python files in repository

...

Found 9 Python files in repository 'security-research-membership-inference-and-differential-privacy' for paper 'Membership Inference Attacks against Machine Learning Models'.
Found 14 Python files in repository 'few-shot-fruit-segmentation-via-transfer-learning' for paper 'Few-Shot Fruit Segmentation via Transfer Learning'.
Found 0 Python files in repository 'shopping-queries-image-dataset' for paper 'Shopping Queries Image Dataset (SQID): An Image-Enriched ESCI Dataset for Exploring Multimodal Learning in Product Search'.
Found 353 Python files in repository 'mulan-code' for paper 'MuLan: Multimodal-LLM Agent for Progressive and Interactive Multi-Object Diffusion'.
Found 7 Python files in repository 'detecting-and-refining-hirise-image-patches-obscured-by-atmospheric-dust' for paper 'Detecting and Refining HiRISE Image Patches Obscured by Atmospheric Dust'.
Found 120 Python files in repository 'reservoirpy' for paper 'Evolving Reservoirs for Meta Reinforcement Learning'.
Found 13 Pyth

Following Github server redirection from /repos/gnaiqing/llmdp to /repositories/673996450


Found 12 Python files in repository 'llmdp' for paper 'Can Large Language Models Design Accurate Label Functions?'.
Found 155 Python files in repository 'gpt-researcher' for paper 'Plan-and-Solve Prompting: Improving Zero-Shot Chain-of-Thought Reasoning by Large Language Models'.
Found 1 Python files in repository 'Unsupervised-Domain-Adaptation-by-Backpropagation-' for paper 'Unsupervised Domain Adaptation by Backpropagation'.
Found 48 Python files in repository 'ilmart' for paper 'ILMART: Interpretable Ranking with Constrained LambdaMART'.
Found 8 Python files in repository 'scarce' for paper 'Learning with Complementary Labels Revisited: The Selected-Completely-at-Random Setting Is More Practical'.
Found 26 Python files in repository 'rl-tools' for paper 'RLtools: A Fast, Portable Deep Reinforcement Learning Library for Continuous Control'.
Found 26 Python files in repository 'rl-tools' for paper 'Data-Driven System Identification of Quadrotors Subject to Motor Delays'.
Found 7 Pyth

Following Github server redirection from /repos/hansenhua/ilid-offline-imitation-learning to /repositories/802911252


Found 1 Python files in repository 'ilid-offline-imitation-learning' for paper 'How to Leverage Diverse Demonstrations in Offline Imitation Learning'.
Found 22 Python files in repository 'adc' for paper 'Resurrecting Old Classes with New Data for Exemplar-Free Continual Learning'.
Found 49 Python files in repository 's3gaussian' for paper '$\textit{S}^3$Gaussian: Self-Supervised Street Gaussians for Autonomous Driving'.
Found 13 Python files in repository 'Unveiling-Linguistic-Regions-in-LLMs' for paper 'Unveiling Linguistic Regions in Large Language Models'.
Found 11 Python files in repository 'average-case-robustness' for paper 'Characterizing Data Point Vulnerability via Average-Case Robustness'.
Found 98 Python files in repository 'mvst' for paper 'Multi-View Spectrogram Transformer for Respiratory Sound Classification'.
Found 56 Python files in repository 'Text-Classification-MSFF-QDConv-Model' for paper 'Multi-Scale Feature Fusion Quantum Depthwise Convolutional Neural Networks f